### 웹툰 이미지를 다운로드하여 로컬에 저장하기

In [1]:
import requests
import os

In [4]:
req_header = {
    #url 참조자
    'referer':'https://comic.naver.com/webtoon/detail?titleId=552960&no=410&amp;weekday=fri'
}

img_urls_list = [
    #이미지 경로
    'https://image-comic.pstatic.net/webtoon/552960/410/20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/552960/410/20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/552960/410/20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_3.jpg'

]

for img_url in img_urls_list:
    res = requests.get(img_url,headers=req_header)
    print(res.status_code)
    # binary data 가져올때 .content 속성사용
    img_data = res.content
    # url에서 파일명만 추출하기
    file_name = os.path.basename(img_url)
    # print(file_name)

    with open(file_name,'wb') as f:
        print(f'Write to file{file_name}({len(img_data)}) bytes')
        f.write(img_data)



200
20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_1.jpg
Write to file20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_1.jpg(162066) bytes
200
20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_2.jpg
Write to file20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_2.jpg(215640) bytes
200
20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_3.jpg
Write to file20220113151757_90bb11af8a341a8bf8a8558fb3d3c121_IMAG01_3.jpg(212535) bytes


### 웹툰의 특정회차의 모든 image 다운로드
* Attribute selector를 사용하여 Jpg파일명들을 모두 추출하여 리스트에 저장하기
* 리스트를 순회하면서 image 다운로드 하기

In [1]:
import requests
from bs4 import BeautifulSoup

main_url = 'https://comic.naver.com/webtoon/detail?titleId=764040&no=54&weekday=fri'
res = requests.get(main_url)


if res.ok:
    soup = BeautifulSoup(res.text,'html.parser')
    img_tags = soup.select("img[src$='.jpg']")#img태그에 src속성에 .jpg
    # print(len(img_tags))
    # print(type(img_tags))
    img_url_list = []

    for img_tag in img_tags:
        # print(img_tag)
        img_url = img_tag['src']
        img_url_list.append(img_url)

    # print(len(img_url_list))
    # print(img_url_list[:4])

In [15]:
#img 디렉토리 생성하기

dir_path = 'img'
import os

#img디렉토리가 없으면
if not os.path.isdir('img'):
    os.mkdir(dir_path)



## 디렉토리 생성하는 2가지 함수
* os.mkdir() / os.makedirs()
1.os.mkdir()은 1개 폴더만 생성
2.os.makedirs()은 a/b/c/ 처럼 하위 폴더 생성

## 리스트 순회하면서 img 다운로드 하기

In [2]:
for idx,img_url in enumerate(img_url_list,1):
    print(f'다운로드번호 {idx} URL = {img_url}')
    req_header = {'referer':main_url}
    res = requests.get(img_url,headers=req_header)
    # print(res)

    if res.ok:
        img_data = res.content
        file_name = os.path.basename(img_url) #파일이름 가져오기
        file_name = 'img/'+file_name

        with open(file_name,'wb') as file:
            file.write(img_data)




다운로드번호 1 URL = https://shared-comic.pstatic.net/thumb/webtoon/764040/thumbnail/thumbnail_IMAG04_8afc9026-2558-49e9-b675-93381d9ef009.jpg


NameError: name 'os' is not defined

#### 파일들이 있는 디렉토리 삭제
#### import shutil 을 사용함

In [21]:
#파일이 있는 디렉토리 삭제
import shutil
import os
dir_path = 'img'

if os.path.exists(dir_path):#img디렉토리가 있다면
    shutil.rmtree(dir_path)

### 웹툰의 타이틀과 특정회차 url을 아규먼트 받아서 다운로드 하는 함수

In [6]:
import requests
from bs4 import BeautifulSoup
import os
import shutil

#img 폴더가 있으면 삭제하고, 없으면 생성한다.
def download_image(title,round_url):
    dir_path = 'img'
    if os.path.exists(dir_path):#img디렉토리가 있다면
        shutil.rmtree(dir_path)
    if not os.path.isdir(dir_path):
        title_path = os.path.join(dir_path,title)

    req_header = {'referer':round_url}
    res = requests.get(round_url,headers=req_header)

    if res.ok:
        # 응답으로 받은 html 텍스트를 파싱하기 위한 파서(Beautifulsoup)객체 생성

        soup = BeautifulSoup(res.text,'html.parser') #res.text 는 html
        img_tags = soup.select("img[src$='.jpg']")#img태그에 src속성에 .jpg가있는 태그만 선택
        # print(soup)

        # 특정회차 url을 referer 헤더로 설정하기
        #
        # req_header = {'referer':round_url}


    for idx,img_tag in enumerate(img_tags,1):

        print(f'------->다운로드번호{idx}')
        img_url = img_tag['src']
        res_img = requests.get(img_url,headers=req_header) #이미지에 대한 요청을 보냄
        if res_img.ok:
            #image binary 데이터 가져오기
            img_data = res_img.content

            # #img\외모지상주의\thumbnail_IMAG04_be987481-7e58-478d-ae1b-c33228ccc658.jpg
           file_name = title_path + '\\' + os.path.basename(img_url)





IndentationError: unindent does not match any outer indentation level (<tokenize>, line 39)

In [56]:
def download_image(title, round_url):
    import requests
    from bs4 import BeautifulSoup
    import os
    import shutil

    # img 폴더가 있으면 삭제하기
    dir_path = 'img'

    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)

    #img 디렉토리가 없으면
    if not os.path.isdir(dir_path):
        #img\외모지상주의
        title_path = os.path.join(dir_path, title)
        print(title_path)
        os.makedirs(title_path)

    # img url 목록을 알아내기 위한 요청을 보내기
    res = requests.get(round_url)
    if res.ok:
        # 응답으로 받은 html텍스트를 파싱하기 위한 파서(BeautifulSoup)객체생성
        soup = BeautifulSoup(res.text, 'html.parser')
        # img태그 중에서 src 속성의 값이 .jpg로 끝나는 태그들만 선택하기
        img_tags = soup.select("img[src$='.jpg']")
        print( len(img_tags), type(img_tags) )

        # 특정회차 url을 referer 헤더로 설정하기
        req_header = { 'referer':round_url }

        for idx, img_tag in enumerate(img_tags,1):
            #img 태그의 src 속성의 값을 추출하기
            print(f'------> 다운로드 번호 {idx}')
            img_url = img_tag['src']

            # jpg image 데이터 요청하기
            res_img = requests.get(img_url, headers=req_header)
            if res_img.ok:
                # image binary 데이터 가져오기
                img_data = res_img.content

                #img\외모지상주의\thumbnail_IMAG04_be987481-7e58-478d-ae1b-c33228ccc658.jpg
                file_name = title_path + '\\' + os.path.basename(img_url)


                with open(file_name, 'wb') as file:
                    print(f'{file_name} ({len(img_data)}) bytes')
                    file.write(img_data)

In [9]:
import requests
upload_files_dict = {
    'img1': open('/Users/jonghwashin/Documents/webscrap_source/data/f1.jpg','rb'),
    'img2': open('/Users/jonghwashin/Documents/webscrap_source/data/f2.jpg','rb'),
    'img3': open('/Users/jonghwashin/Documents/webscrap_source/data/f3.jpg','rb'),
}
upload_files_dict

{'img1': <_io.BufferedReader name='/Users/jonghwashin/Documents/webscrap_source/data/f1.jpg'>,
 'img2': <_io.BufferedReader name='/Users/jonghwashin/Documents/webscrap_source/data/f2.jpg'>,
 'img3': <_io.BufferedReader name='/Users/jonghwashin/Documents/webscrap_source/data/f3.jpg'>}

### image file upload
* https://httpbin.org/post URL로 요청을 보내서 다운로드 받은 img 파일을 업로드 하기
* requests.post() 함수를 사용하고, files 속성에 image data를 지정한다.
* files – (optional) Dictionary of ‘filename’: file-like-objects for multipart encoding upload.

In [7]:
url = 'http://httpbin.org/post'
res = requests.post(url,files = upload_files_dict)
# print(res.status_code)
# print(res)
res_data = res.json()
# print(res_data)


NameError: name 'upload_files_dict' is not defined